In [1]:
import numpy as np
import pandas as pd

In [4]:
from pytorch_inferno.data import *

In [43]:
from pytorch_inferno.pseudodata import paper_sig, paper_bkg, PseudoData

In [36]:
bs = 2000
data, test = get_paper_data(200000, bs=bs, n_test=1000000)

In [ ]:
for b in data.trn_dl:
    print(b[0].shape)

In [44]:
sig_trn = PseudoData(paper_sig, 1).sample(1000)

In [48]:
sig_trn[1].shape

(1000, 1)

In [11]:
signal = pd.read_hdf("/home/centos/data/bdt_rs5/TTJets_signal.h5")
bkg = pd.read_hdf("/home/centos/data/bdt_rs5/QCD.h5")

In [12]:
import random
def train_test_split(df, n = 5000):

    df.reset_index(drop=True, inplace=True)
    indices = list(df.index)
    train_id = random.sample(indices, n)
    test_id = list(set(indices) - set(train_id))
    train_mask = df.index.isin(train_id)
    test_mask = df.index.isin(test_id)
    cond = [train_mask, test_mask]
    choice = ["train", "test"]
    df['train_flag'] = np.select(cond, choice)

In [82]:
def get_cmsopen_data(n_sig = 4000, n_bkg = 4000, bs=256):
        
    #signal = samples["TTJets_signal"] #samples["TTJets_centJER_signal"] #samples["TTJets_signal_central"]
    #bkg = samples["QCD"]
    
    train_test_split(signal, n_sig)
    train_test_split(bkg, n_bkg)
    
    print(list(signal))
    
    signal["label"] = 0
    bkg["label"] = 1
    
    signal_train = signal[signal["train_flag"] == "train"]
    signal_train["weights"] = signal_train['trigger_weight'] * signal_train['btag_weight1']
    signal_train["weights"] = signal_train["weights"] * (1. / np.mean(signal_train["weights"]))
    signal_test = signal[signal["train_flag"] == "test"]
    bkg_train = bkg[bkg["train_flag"] == "train"]
    bkg_train["weights"] = bkg_train['btag_weight2']
    bkg_train["weights"] = bkg_train["weights"] * (1. / np.mean(bkg_train["weights"]))
    bkg_test = bkg[bkg["train_flag"] == "test"]
    
    train_data = pd.concat([signal_train, bkg_train], axis=0)
    test_data = pd.concat([signal_test, bkg_test], axis=0)
    
    features = ['ht', 'aplanarity', 'sphericity', 'chargeEta', 'MET_met', 'deltaPhiTauMet', 'mt', 'mTauJet']

    X_train = train_data[features].values
    y_train = train_data["label"].values.reshape((-1,1))
    weights_train = train_data["weights"]
    
    X_test = test_data[features].values
    y_test = test_data["label"].values.reshape((-1,1))
    #weights_test= test_data["weights"]

    
    trn = (X_train, y_train)
    val = (X_test, y_test)
    
    trn_dl = WeightedDataLoader(DataSet(*trn), batch_size=bs, shuffle=True, drop_last=True)
    val_dl = WeightedDataLoader(DataSet(*val), batch_size=bs, shuffle=True)
    data = DataPair(trn_dl, val_dl)
    
    return data

In [83]:
data = prepare_data(n_sig = 4000, n_bkg = 4000)

['MET_met', 'event', 'run', 'luminosityBlock', 'HLT_QuadJet40_IsoPFTau40', 'HLT_QuadJet45_IsoPFTau45', 'PV_npvs', 'genEvent_tmeme', 'norm', 'norm_up', 'norm_down', 'trigger_weight', 'trigger_weight_up', 'trigger_weight_down', 'trigger_is40', 'isSignal', 'isBkg', 'btag_weight1', 'btag_weight1_up', 'btag_weight1_down', 'btag_weight2', 'btag_weight2_up', 'btag_weight2_down', 'pdf', 'pdf_up', 'pdf_down', 'h', 'ht', 'h_jet', 'ht_jet', 'chargeEta', 'mTauJet', 'mt', 'deltaPhiTauMet', 'nJets', 'aplanarity', 'sphericity', 'train_flag', 'label', 'bdt']


/home/centos/env/lib64/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/centos/env/lib64/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/home/centos/env/lib64/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [74]:
trn_dl = WeightedDataLoader(DataSet(*trn), batch_size=256, shuffle=True, drop_last=True)
val_dl = WeightedDataLoader(DataSet(*val), batch_size=256, shuffle=True)

In [78]:
data = DataPair(trn_dl, val_dl)

In [79]:
for b in data.trn_dl:
    print(b[0].shape)

torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])


In [69]:
for b in trn_dl:
    print(b[0].shape)

torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])
torch.Size([256, 8])


In [ ]:
def get_cmsopen_data(n_sig = 5000, bs=2000):
    
    train_test_split(df, n_sig)

In [3]:
def get_paper_data(n, bs=2000, n_test):
    
    n,n_test = n//2,n_test//2
    sig_trn = PseudoData(paper_sig, 1).sample(n)
    bkg_trn = PseudoData(paper_bkg, 0).sample(n)
    sig_val = PseudoData(paper_sig, 1).sample(n)
    bkg_val = PseudoData(paper_bkg, 0).sample(n)

    trn = (np.vstack((sig_trn[0],bkg_trn[0])),np.vstack((sig_trn[1],bkg_trn[1])))
    val = (np.vstack((sig_val[0],bkg_val[0])),np.vstack((sig_val[1],bkg_val[1])))

    trn_dl = WeightedDataLoader(DataSet(*trn), batch_size=bs, shuffle=True, drop_last=True)
    val_dl = WeightedDataLoader(DataSet(*val), batch_size=2*bs, shuffle=True)
    data = DataPair(trn_dl, val_dl)
    if n_test <= 0: return data

    sig_tst = PseudoData(paper_sig, 1).sample(n_test)
    bkg_tst = PseudoData(paper_bkg, 0).sample(n_test)
    tst = (np.vstack((sig_tst[0],bkg_tst[0])),np.vstack((sig_tst[1],bkg_tst[1])))
    tst_dl = WeightedDataLoader(DataSet(*tst), batch_size=2*bs)
    return data, tst_dl

,MET_met,event,run,luminosityBlock,HLT_QuadJet40_IsoPFTau40,HLT_QuadJet45_IsoPFTau45,PV_npvs,genEvent_tmeme,norm,norm_up,...,chargeEta,mTauJet,mt,deltaPhiTauMet,nJets,aplanarity,sphericity,train_flag,label,bdt
0,120.194692,121740,1,592191,1,1,7,10000,0.012424,0.013182,...,0.098549,361.638426,9.238372,0.114702,4,0.050326,0.517126,train,0,0.961372
1,143.947763,276237,1,207629,1,1,8,10000,0.012424,0.013182,...,-0.084550,894.162165,24.981896,0.268102,5,0.039740,0.154885,test,0,0.943711
2,136.312132,228988,1,67444,1,1,14,10000,0.012424,0.013182,...,-0.781336,626.741711,16.462205,0.113007,5,0.034599,0.438753,test,0,0.950682
3,56.325748,208469,1,69042,1,1,4,10000,0.012424,0.013182,...,-0.310569,555.370084,42.891924,0.796644,4,0.026440,0.125787,test,0,0.861102
4,186.922750,427218,1,69772,1,1,5,10000,0.012424,0.013182,...,0.244730,685.191449,37.803811,0.347520,5,0.174086,0.409456,train,0,0.958536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43565,86.944997,449767,1,391578,1,1,10,10000,0.012424,0.013182,...,0.340606,837.191196,55.515881,0.586703,6,0.067183,0.210311,test,0,0.938444
43566,78.446850,180814,1,392348,1,1,15,10000,0.012424,0.013182,...,-0.882663,496.005885,48.106776,0.575564,4,0.052211,0.424178,test,0,0.964670
43567,43.084267,98087,1,412076,1,1,9,10000,0.012424,0.013182,...,-0.199707,347.225131,35.460463,0.623681,4,0.206531,0.577335,test,0,0.748297
43568,182.012304,172294,1,418992,1,1,7,10000,0.012424,0.013182,...,1.915965,930.673878,1.349138,0.010976,4,0.065533,0.481673,test,0,0.784848
